# Get Current Eclipse Template
This notebook is intended to extract *structure information* from the current 
Eclipse templates.

In Version 15.6, the Eclipse Templates are located on the Varian Image server in:
[Eclipse Templates](file:\\ARIAIMGPV1\va_data$\ProgramData\Vision\Templates).
Under this folder are sub-folders containing *.xml* formatted templates.  
The folders containing structure information are:
- objective
- protocol
- structure

The .xml files in these folders can be coppied into corresponding folders in 
the *Reference* directory.

**NOTE:** These templates should ***not*** be modified directly.  
However copies can be made, modified and re-imported if batch-level changes are 
desired.

**NOTE:** The templates are expected to follow Version 1.2 Eclipse Template 
format (described below).  Failure to comply with that format, or the presence 
of non-Eclipse Template .xml files in the folders may case the program to crash. 



## Structure Template .xml File Format
- The structures template contains two primary elements:
  - Preview
  - Structures
  
![StructureTemplate](\Template%20ScreenShots\Structure%20Template%20Overview.png)


### Preview Element
- All Preview information is contained in attributes.
  
![Preview](\Template%20ScreenShots/Preview.png)
> ```
>  <Preview ID="Prostate SIB 68 in 25"
>           Type="Structure"
>           ApprovalStatus="Approved"
>           Diagnosis=""
>           TreatmentSite=""
>           Description="Prostate Single Integrated Boost 68 Gy in 25 fractions"
>           AssignedUsers="gsal"
>           LastModified=" March 04 2020 15:45:33:973"
>           ApprovalHistory="gsal Created [ March 04 2020 15:45:33:964 ]"/>
> ```

### Structures Element
- The *Structures* element does not contain any attributes.
- The *Structures* element contains multiple *Structure* elements.

### **Structure** Element
- The **Structure** element contains two attributes:
  - ID
  - Name
- The **Structure** element contains 12 sub elements:

|ELement        | Description                      |
|---------------|----------------------------------|
|Identification |Structure Volume and Dictionary references<br>(See below for more details)|
|TypeIndex |Always set to *2*|
|ColorAndStyle |Reference to display settings in the *Material* database table<br>(See below for more details)|
|SearchCTLow<br>SearchCTHigh|Minimum and Maximum expected HU values<br>(See below for more details)|
|DVHLineStyle<br>DVHLineColorDVH<br>DVHLineWidth|DVH to display settings<br>(See below for more details)|
|EUDAlpha<br>TCPAlpha<br>TCPBeta<br>TCPGamma| *Not Used*|

![Structures](\Template%20ScreenShots/Structure%20Content.png)


### Identification Element
- The **Identification** element contains 5 sub elements:

|ELement        | Description                      |
|---------------|----------------------------------|
|StructureCode  |The Structure Dictionary Reference <br>[(See below for more details)][StructureCode]|
|VolumeCode     |A disease code<br>Usually blank|
|VolumeCodeTable|The disease code table referenced<br>Usually blank|
|VolumeType     |A Structure Volume type<br>[(See the Structure Volume Types table below)][Structure Volume Types]|
|VolumeID       |Usually blank|

![Identification](\Template%20ScreenShots/Identification%20DPV.png)

[Structure Volume Types]:
### Structure Volume Types:
| VolumeType     | Description                                                  | DicomType      | VolumeTypeSer |
|----------------|--------------------------------------------------------------|----------------|---------------|
| GTV            | Gross Tumor Volume                                           | GTV            | 2             |
| CTV            | Clinical Target Volume                                       | CTV            | 3             |
| PTV            | Planning Target Volume                                       | PTV            | 4             |
| PTV Phase 2    | Reduced Planning Target Volume for second phase of treatment | PTV            | 34            |
| Nodes          | Nodes potentially at risk                                    | ORGAN          | 10014         |
| BODY           | Body                                                         | EXTERNAL       | 6             |
| Organ          | Region Of Interest                                           | ORGAN          | 7             |
| Avoidance      | Organ At Risk                                                | AVOIDANCE      | 5             |
| Control        | Dose Optimization Region                                     | CONTROL        | 15            |
| Treated Volume | Treated Volume                                               | TREATED_VOLUME | 8             |
| Irrad Volume   | Irradiated Volume                                            | IRRAD_VOLUME   | 9             |
| Support        | Patient Support Device                                       | SUPPORT        | 12            |
| Fixation       | Patient Fixation Device                                      | FIXATION       | 13            |
| Contrast Agent | Contrast Agent                                               | CONTRAST_AGENT | 10            |
| Cavity         | Cavity                                                       | CAVITY         | 11            |
| Dose Region    | Dose Reference Region                                        | DOSE_REGION    | 14            |
| Bolus          | [NULL]                                                       | BOLUS          | 32            |
| None           | None                                                         | [NULL]         | 1             |
| Marker         | [NULL]                                                       | [NULL]         | 33            |
| Reference      | [NULL]                                                       | [NULL]         | 10002         |


[StructureCode]:
### StructureCode
- StructureCode contains three attributes that reference one of three [structure dictionaries][Dictionary Schemes].
  - Code
  - CodeScheme
  - CodeSchemeVersion

![StructureCode](\Template%20ScreenShots/StructureCode%20DPV.png)

> ```
>        <StructureCode Code="Treated Volume"
>                       CodeScheme="99VMS_STRUCTCODE"
>                       CodeSchemeVersion="1.0"/>
> ```


[Dictionary Schemes][Dictionary Schemes]
### Structure Dictionary Schemes
[Dictionary Schemes]:
| Designator       | Version  |CommonName                    | ResponsibleOrganization               | DICOMCodeSchemeSer |
|------------------|----------|------------------------------|---------------------------------------|--------------------|
| 99VMS_STRUCTCODE | 1.0      |Structure Codes               | Varian Medical Systems                | 15                 |
| FMA              | 3.2      |Foundational Model of Anatomy | University of Washington              | 16                 |
| RADLEX           | 3.8      |RadLex                        | Radiological Society of North America | 17                 |


### SearchCT
- The SearchCTLow and SearchCTHigh elements refer to HU values for auto segmentation.  
- If no values are supplied the elements will contain an attribute: `xsi:nil="true"`

![SearchCT](\Template%20ScreenShots/SearchCT.png)

> ```
> <SearchCTLow>-350</SearchCTLow>
> <SearchCTHigh>-50</SearchCTHigh>
> ```
> ...
> ```
> <SearchCTLow xsi:nil="true"/>
> <SearchCTHigh xsi:nil="true"/>
> ```

### DVH Style Parameters
- The elements DVHLineStyle, DVHLineColor, DVHLineWidth dictate the line 
  formatting for DVH curves.
```
<DVHLineStyle>0</DVHLineStyle>
<DVHLineColor>-16777216</DVHLineColor>
<DVHLineWidth>3</DVHLineWidth>
```

## Setup


### Imports

In [21]:
from typing import Tuple, Dict, List

from pathlib import Path
import xml.etree.ElementTree as ET

import pandas as pd
import xlwings as xw

import varian_query as vq


## Functions

### Convert Hex color values to an RGB tuple

In [22]:
def hex_to_rgb(hex_b):
    '''`Convert a 6-byte Hex color code to RGB values.`

    Will accept 6-byte or 8-byte float or bytes objects and return a string 
    representation of an RGB tuple. if an 8-byte value (with alpha) is provided, 
    the alpha portion will be dropped.

    Args:
        hex_b (float, bytes): A 6-byte Hex color code

    Returns:
        str: A string representation of the corresponding RGB tuple
    '''
    # Skip empty values
    if not hex_b:
        return hex_b
    # Convert the Hex value to a string of hex characters.
    # This function works the same for float or bytes.
    # tio accept integers, the call would have to be hex(hex_b).
    hex_s = hex_b.hex()
    # convert hex character pairs into integers
    rgb_list = [int(hex_s[i:i+2], 16)  for i in (0, 2, 4)]
    # Revers the order because Red is the lowest (right most) values in the hex 
    # number.
    rgb_list.reverse()
    # Convert the RGB list into a string representation of a tuple for easy 
    # visualization.
    rgb = tuple(rgb_list)
    rgb_str = str(rgb)
    return rgb_str

### Functions to obtain reference information from the Varian database

In [23]:
def get_structure_dictionary(aria_con: 'ODBC.Connection', 
                             sql_path: Path)->pd.DataFrame:
    '''Get the Varian Structure Dictionary table from the Varian database.

    The table contain the following columns: 
        Code
        Label
        Synonyms
        Scheme
        Version
        Default Structure Id
        Protected
    Only 'Active' dictionary entries are included.

    Args:
        aria_con (ODBC.Connection): Varian ARIA Database Connection
        sql_path (Path): Path to the folder containing the sql scripts.

    Returns:
        pd.DataFrame: The local Varian Structure Dictionary table.
    '''
    query_file = sql_path / 'StructureDictionary.sql'
    structure_dict = vq.run_query(aria_con, query_file)    
    structure_dict.set_index('Code', inplace=True)
    active_items = structure_dict.Active.str.contains('TRUE')
    selected_columns = ['Label', 'Synonyms', 'Scheme', 'Version', 
                        'Default  Structure Id','Protected']
    structure_dict = structure_dict.loc[active_items, selected_columns].copy()
    return structure_dict

In [24]:
def get_structure_volumes(aria_con: 'ODBC.Connection', 
                          sql_path: Path)->pd.DataFrame:
    '''Get the Structure Volume table from the Varian database.

    The table contain the following columns: 
        VolumeType
        VolumeDescription
        DicomStructureType
    Only 'Active' volume entries are included.

    Args:
        aria_con (ODBC.Connection): Varian ARIA Database Connection
        sql_path (Path): Path to the folder containing the sql scripts.

    Returns:
        pd.DataFrame: The local Structure Volume table table.
    '''
    query_file = sql_path / 'StructureVolume.sql'

    structure_volume = vq.run_query(aria_con, query_file)
    structure_volume.set_index('VolumeType', inplace=True)
    active_items = structure_volume.VolumeTypeStatus.str.contains('Active')
    selected_columns = ['VolumeDescription', 'DicomStructureType', 
                        'StructureOverlap']
    structure_volume = structure_volume.loc[active_items, selected_columns].copy()
    return structure_volume

In [25]:
def get_colors(aria_con: 'ODBC.Connection', sql_path: Path)->pd.DataFrame:
    '''Get the Structure Colors (Material) table from the Varian database.

    The table contain the following columns: 
        ColorId
        ColorName
        2D_Color
        2D_View
        2D_Translucency
        3D_Color
        3D_View
        3D_Translucency

    Args:
        aria_con (ODBC.Connection): Varian ARIA Database Connection
        sql_path (Path): Path to the folder containing the sql scripts.

    Returns:
        pd.DataFrame: The local Structure Volume table table.
    '''
    query_file = sql_path / 'color_ref.sql'
    color_ref = vq.run_query(aria_con, query_file)
    # Convert Color Hex values to RGB strings.
    color_ref['2D_Color'] = color_ref['2D_Color'].apply(hex_to_rgb)
    color_ref['3D_Color'] = color_ref['3D_Color'].apply(hex_to_rgb)
    color_ref.set_index('ColorId', inplace=True)
    return color_ref

### Extract Information from structure template *.xml* files

In [26]:
def get_structure_info(template_id: str, 
                       structures: ET.Element)->List[Dict[str, str]]:
    '''Extract relevant information from each structure definition element.

    For each structure element in the supplied structures element, text 
    information is extracted from sub-elements. The information from the 
    structure is placed in a dictionary with the supplied template_id as an 
    additional item. A list of all of the dictionaries is returned.
    
    The following <Structure> attributes are extracted to the dictionary:
        ID
        Name
    The following elements' text is extracted
        <Structure>.<TemplateId>
        <Structure>.<ColorAndStyle>
        <Structure>.<DVHLineStyle>
        <Structure>.<DVHLineColor>
        <Structure>.<DVHLineWidth>
        <Structure>.<VolumeID>
        <Structure>.<Identification>.<VolumeID>
        <Structure>.<Identification>.<VolumeCode>
        <Structure>.<Identification>.<VolumeType>
        <Structure>.<Identification>.<VolumeCodeTable>
    The following <Structure>.<Identification>.<StructureCode> attributes are 
    also extracted to the dictionary:
        Code
        CodeScheme
        CodeSchemeVersion
    
    Args:
        template_id (str): The name of the structure template.
        structures (ET.Element): A structures element from the structure 
            template .xml file 

    Returns:
        List[Dict[str, str]]: All of the structure information extracted from 
        the supplied element.
    '''
    structure_dict_list = []
    for struct in structures:
        # Get the Identification sub-element.  It does not contain attributes, 
        # but text from sub-elements of Identification are desired.
        struct_ident = struct.find('Identification')
        # Initialize the dictionary with the ID and Name attributes of the 
        # structure element.
        structure_dict = struct.attrib
        # Extract text from the desired sub-elements and create dictionary items 
        # with the sub-element as the key.
        structure_dict['TemplateId'] = template_id
        structure_dict['ColorAndStyle'] = struct.findtext('ColorAndStyle')
        structure_dict['DVHLineStyle'] = struct.findtext('DVHLineStyle')
        structure_dict['DVHLineColor'] = struct.findtext('DVHLineColor')
        structure_dict['DVHLineWidth'] = struct.findtext('DVHLineWidth')
        # Extract text from the desired sub-elements of the Identification sub-element
        # and create dictionary items with the sub-element as the key.
        structure_dict['VolumeID'] = struct_ident.findtext('VolumeID')
        structure_dict['VolumeCode'] = struct_ident.findtext('VolumeCode')
        structure_dict['VolumeType'] = struct_ident.findtext('VolumeType')
        structure_dict['VolumeCodeTable'] = struct_ident.findtext('VolumeCodeTable')
        # Add the <Identification>.<StructureCode> attributes to the dictionary.
        structure_dict.update(struct_ident.find('StructureCode').attrib)
        # Add the dictionary of structure information to the list.
        structure_dict_list.append(structure_dict)
    return structure_dict_list


def get_struct_templates(structure_xml_folder: Path)-> Tuple[pd.DataFrame]:
    '''Extract Information from all Structure Templates in a folder.

    Identifies all .xml files in a folder and tries to extract structure .
    information from them.

    Args:
        structure_xml_files (Path): Full path to the directory containing the 
        structure template .xml files.

    Returns:
        Tuple[pd.DataFrame]: Tables containing the structure information for 
            all templates and the general information from each template.
    '''
    template_dicts = []
    structure_lists = []
    # Extract structure and template info for each .xml file in the given 
    # directory.  It assumes that all .xml file sin the folder are Version 1.2 
    # structure templates.  If not, try .. except statements will have to be 
    # added to catch the invalid files.
    for struct_xml in structure_xml_folder.glob('*.xml'):
        # Load the xml data from the file.
        tree = ET.parse(struct_xml)
        root = tree.getroot()
        # Get the template information from the Preview element.
        preview = root.find('Preview')
        template_dict = {}
        template_dict.update(preview.attrib)
        template_dict['FileName'] = struct_xml.name
        template_dicts.append(template_dict)
        # Get the structure information from the template.
        template_id = preview.attrib['ID']
        structures = root.find('Structures')
        structure_data = get_structure_info(template_id, structures)
        structure_lists.extend(structure_data)
    # Build tables from the extracted information.
    template_data = pd.DataFrame(template_dicts)
    structure_data = pd.DataFrame(structure_lists)
    return template_data, structure_data


def get_structures(structure_xml_folder: Path, aria_con: 'ODBC.Connection', 
                   sql_path: Path)->pd.DataFrame:
    '''Build a table of structure information from all structure templates in a 
    folder.

    The structure information from each structure template is combined with 
    general template information, structure dictionary information color tables,
    and Volume type information to produce a large flat-file table that can be 
    analyzed with the Excel Pivot tool.

    Args:
        structure_xml_folder (Path): Full path to the directory containing the 
        structure template .xml files.
        aria_con (ODBC.Connection): Varian ARIA Database Connection
        sql_path (Path): Path to the folder containing the sql scripts.

    Returns:
        pd.DataFrame: A flat-file table containing structure information from 
            each in the folder.
    '''
    # Extract Information from all structure templates in a folder.
    template_data, structure_data = get_struct_templates(structure_xml_folder)
    # Select the desired columns from the template information and rename the 
    # columns appropriately.
    column_names = {
        'Type': 'Template Type', 
        'ApprovalStatus': 'TemplateStatus', 
        'Diagnosis': 'TemplateDiagnosis', 
        'TreatmentSite': 'TemplateSite', 
        'Description': 'TemplateDescription',
        'FileName': 'TemplateFileName'
        }
    template_data.set_index('ID', inplace=True)
    selected_rows = template_data.ApprovalStatus.isin(['Approved', 'Reviewed'])
    template_data.rename(columns=column_names, inplace=True)
    selected_columns = list(column_names.values())
    template_data = template_data.loc[selected_rows, selected_columns].copy()
    # Merge the template information with each row of the structure information.
    # The result is a large flat-file table that can be analyzed with the 
    # Excel Pivot tool.
    structure_data = structure_data.join(template_data, on='TemplateId', 
                                         how='inner')    
    # Add Structure Dictionary reference information.
    structure_dict = get_structure_dictionary(aria_con, sql_path)
    structure_data = structure_data.join(structure_dict, on='Code')
    # Add Color Table reference information
    color_ref = get_colors(aria_con, sql_path)
    structure_data = structure_data.join(color_ref, on='ColorAndStyle')
    # Add volume type reference information
    structure_volume = get_structure_volumes(aria_con, sql_path)
    structure_data = structure_data.join(structure_volume, on='VolumeType')
    return structure_data

In [27]:
def get_obj_structures(template_id: str, 
                             structures: ET.Element)->List[Dict[str, str]]:
    '''Extract relevant information from each objective structure element.

    For each Objective element in the supplied StructureObjectives element, text 
    information is extracted from sub-elements. The information from the 
    structure is placed in a dictionary with the supplied template_id as an 
    additional item, with the key: 
        TemplateId
    A list of all of the dictionaries is returned.    
    The following <ObjectivesOneStructure> attributes are extracted to the 
    dictionary:
        ID
        Name
        SurfaceOnly
    The following elements' text is extracted
        <ObjectivesOneStructure>.<Distance>
        <ObjectivesOneStructure>.<Color>
        <ObjectivesOneStructure>.<StructureTarget>.<VolumeID>
        <ObjectivesOneStructure>.<StructureTarget>.<VolumeCode>
        <ObjectivesOneStructure>.<StructureTarget>.<VolumeType>
        <ObjectivesOneStructure>.<StructureTarget>.<VolumeCodeTable>
    The following <ObjectivesOneStructure>.<SamplePoints attributes are 
    also extracted to the dictionary:
        xsi:nil
    In addition, the number of objective associated with the structure are 
    counted and added to the dictionary as:
        ObjectiveCount
    
    Args:
        template_id (str): The name of the objective template.
        structures (ET.Element): An ObjectivesAllStructures element from the 
            objective template .xml file 

    Returns:
        List[Dict[str, str]]: All of the objective structure information 
        extracted from the supplied element.
    '''
    structure_dict_list = []
    for struct in structures:
        struct_ident = struct.find('StructureTarget')
        struct_obj = struct.find('StructureObjectives')
        num_objective = len(struct_obj.findall('Objective'))            
        structure_dict = struct.attrib
        structure_dict['TemplateId'] = template_id
        structure_dict['ObjectiveCount'] = num_objective
        structure_dict['Color'] = struct.find('Color').text
        structure_dict['VolumeID'] = struct_ident.find('VolumeID').text
        structure_dict['VolumeCode'] = struct_ident.find('VolumeCode').text
        structure_dict['VolumeType'] = struct_ident.find('VolumeType').text
        structure_dict['VolumeCodeTable'] = struct_ident.find('VolumeCodeTable').text
        structure_dict['Distance'] = struct.find('Distance').text
        structure_dict.update(struct.find('SamplePoints').attrib)
        structure_dict_list.append(structure_dict)
    return structure_dict_list
        
        
def get_objective_templates(structure_xml_files):
    '''Extract Information from all Objective Templates in a folder.

    Identifies all .xml files in a folder and tries to extract structure .
    information from them.

    Args:
        structure_xml_files (Path): Full path to the directory containing the 
        structure template .xml files.

    Returns:
        Tuple[pd.DataFrame]: Tables containing the structure information for 
            all templates and the general information from each template.
    '''
    template_dicts = []
    structure_lists = []
    # Extract structure and template info for each .xml file in the given 
    # directory.  It assumes that all .xml file sin the folder are Version 1.2 
    # objective templates.  If not, try .. except statements will have to be 
    # added to catch the invalid files.    
    for struct_xml in structure_xml_files.glob('*.xml'):
        tree = ET.parse(struct_xml)
        root = tree.getroot()
        preview = root.find('Preview')
        template_dict = {}
        template_dict.update(preview.attrib)
        template_dict['FileName'] = struct_xml.name        
        template_dicts.append(template_dict)
        template_id = preview.attrib['ID']
        structures = root.find('ObjectivesAllStructures')
        structure_dict_list = get_obj_structures(template_id, structures)
        structure_lists.extend(structure_dict_list)
    template_data = pd.DataFrame(template_dicts)
    structure_data = pd.DataFrame(structure_lists) 
    return template_data, structure_data


def get_objective_structures(structure_xml_folder: Path)->pd.DataFrame:
    '''Build a table of structure information from all objective templates in a 
    folder.

    The template information is combined with structure information from each 
    objective template to produce a large flat-file table that can be analyzed 
    with the Excel Pivot tool.

    Args:
        structure_xml_folder (Path): Full path to the directory containing the 
        structure template .xml files.
        aria_con (ODBC.Connection): Varian ARIA Database Connection
        sql_path (Path): Path to the folder containing the sql scripts.

    Returns:
        pd.DataFrame: A flat-file table containing structure information from 
            each objective template in the folder.
    '''
    # Extract Information from all objective templates in a folder.
    template_data, structure_data = get_objective_templates(structure_xml_folder)
    # Select the desired columns from the template information and rename the 
    # columns appropriately.
    column_names = {
        'Type': 'Template Type', 
        'ApprovalStatus': 'TemplateStatus', 
        'Diagnosis': 'TemplateDiagnosis', 
        'TreatmentSite': 'TemplateSite', 
        'Description': 'TemplateDescription',
        'FileName': 'TemplateFileName'
        }
    template_data.set_index('ID', inplace=True)
    selected_rows = template_data.ApprovalStatus.isin(['Approved', 'Reviewed'])
    template_data.rename(columns=column_names, inplace=True)
    selected_columns = list(column_names.values())
    template_data = template_data.loc[selected_rows, selected_columns].copy()
    # Merge the template information with each row of the structure information.
    # The result is a large flat-file table that can be analyzed with the 
    # Excel Pivot tool.
    structure_data = structure_data.join(template_data, on='TemplateId',
                                         how='inner')    
    column_names = {
        'TemplateId': 'TemplateId', 
        'ID': 'StructureId', 
        'NAME': 'StructureName',
        'SurfaceOnly': 'StructureSurface', 
        'ObjectiveCount': 'NumberOfObjectives', 
        'Color': 'StructureColor',
        'VolumeID': 'StructureVolumeID', 
        'VolumeCode': 'StructureVolumeCode', 
        'VolumeType': 'StructureVolumeType', 
        'VolumeCodeTable': 'StructureVolumeCodeTable',
        'Distance': 'Distance',
        '{http://www.w3.org/2001/XMLSchema-instance}nil': 'NoSamplePoints'
        }
    structure_data.rename(columns=column_names, inplace=True)
    return structure_data





### Connect to Varian Database

In [28]:
DBSERVER = 'ARIADBPV1'
SYSTEMDB = 'VARIAN'
aria_con = vq.connect(DBSERVER, SYSTEMDB)

### Set Paths

In [29]:
# Selects one level up from the current working directory as the starting point.
base_path = Path.cwd() / '..'
base_path = base_path.resolve()

# Assumes that sql scripts are stored in a subdirectory *src*
sql_path = base_path / 'src'

# Save resulting spreadsheets to the *Structures and Templates* subdirectory
save_path = base_path / r'Structures and Templates'

# Paths to the folders containing the copied Structure and Objective templates
structure_xml_files = base_path / r'Reference\Current structure Templates'
objective_xml_files = base_path / r'Reference\Current Objective Templates'

#### ARIA paths

In [30]:
server_template_path = Path(r'\\ARIAIMGPV1\va_data$\ProgramData\Vision\Templates')

aria_structure_xml_files = server_template_path / r'structure'
aria_objective_xml_files = server_template_path / r'objective'

### Make a copy of the Eclipse templates so that we don't work with the originals

#### Delete old templates from the target directory

In [ ]:
!del "{structure_xml_files}\*.xml" 
!del "{objective_xml_files}\*.xml" 

#### Copy templates from the Eclipse file system

In [ ]:
!xcopy "{aria_structure_xml_files}" "{structure_xml_files}" /Y
!xcopy "{aria_objective_xml_files}" "{objective_xml_files}" /Y

### Rename Templates

In [33]:
def fix_label(name:str)->str:
    '''Convert template IDs to legal file names.
    
    Convert ID like: VMAT H&N 66/33 into legal file name like VMAT HN 66 in 33
    Remove illegal '&' characters from name.
    Replace '/' with ' in '.
    Add '.xml' to end.

    Args:
        name (str): Template ID

    Returns:
        str: Legal file name
    '''
    # Remove illegal '&' characters from name
    name = name.replace('&', '')
    # Replace '/' with ' in '
    name = name.replace('/', ' in ')
    # Add '.xml' to end.
    name = name + '.xml'
    return name

In [34]:
def rename_templates(template_path: Path):    
    '''Rename the Eclipse template with it's ID.

    Extract template ID, clean the ID to remove or replace invalid characters, 
    rename the template from it's generic numerical name to the cleaned ID.
    
    Args:
        template_path (Path): Path to directory containing Eclipse 
            templates to be renamed
    '''
    for template_file in template_path.glob('*.xml'):
        template_text = template_file.read_text()
        root = ET.fromstring(template_text)
        preview_el = root.find(r'.//Preview')
        template_id = preview_el.get('ID')
        new_name = fix_label(template_id)
        new_path = template_file.parent / new_name
        template_file.rename(new_path)
        print(f'Found Template: {new_name}')


In [ ]:
rename_templates(structure_xml_files)
rename_templates(objective_xml_files)

In [36]:
structure_data = get_structures(structure_xml_files, aria_con, sql_path)

obj_structure_data = get_objective_structures(objective_xml_files)

In [37]:
structure_dict = get_structure_dictionary(aria_con, sql_path)
color_ref = get_colors(aria_con, sql_path)
structure_volumes = get_structure_volumes(aria_con, sql_path)

In [38]:
save_file = save_path / 'Current Structures & Templates.xlsx'
wb = xw.Book()
wb.save(save_file)

sheet = wb.sheets.add('Color Table')
xw.view(color_ref, sheet=sheet)

sheet = wb.sheets.add('Structure Dictionary')
xw.view(structure_dict, sheet=sheet)

sheet = wb.sheets.add('Structure Volumes')
xw.view(structure_volumes, sheet=sheet)

#sheet = wb.sheets.add('Structure Templates')
#xw.view(template_data, sheet=sheet)

#sheet = wb.sheets.add('Objective Template')
#xw.view(obj_template_data, sheet=sheet)

sheet = wb.sheets.add('Objective Structures')
xw.view(obj_structure_data, sheet=sheet)

sheet = wb.sheets.add('Structures')
xw.view(structure_data, sheet=sheet)

wb.save(save_file)